In [1]:
import sys
from essential_generators import DocumentGenerator
import io
import re
import numpy as np
import nltk
nltk.download('punkt')
import os
import math


def gen_data():
    """
    Data generator
    """
    gen = DocumentGenerator()
    for i in range(1, 1101):
        with io.open(f"sample_text/text_{i}.txt", "w", encoding="utf-8") as f:
            # for j in range(1, 10):
            p = gen.paragraph()
            f.write(p)
            f.close()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\surja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def create_dict():
    dictionary={}
    for i in range(1, 1101):
        with io.open(f"sample_text/text_{i}.txt", "r", encoding="utf-8") as f:
            data = f.read()
            f.close()
            data = data.lower()
            data = re.sub(r'[^\w\s]','',data)
            data = re.sub('[0-9]','',data)
            tokenized_words=nltk.word_tokenize(data)
            for word in tokenized_words:
                if word in dictionary.keys():
                    dictionary[word]+=1
                else:
                    dictionary[word]=1
    return dictionary

In [3]:
dictionary = create_dict()
len(dictionary)

14319

In [4]:
def bag_of_words_vector(file):
    vector_dict={}
    for w in dictionary.keys():
        vector_dict[w]=0
    f = io.open(file,encoding="utf8")
    data = f.read()
    f.close()
    data = data.lower()
    data = re.sub(r'[^\w\s]','',data)
    data = re.sub('[0-9]','',data)
    words=nltk.word_tokenize(data)
    for w in words:
        vector_dict[w]+=1
    return vector_dict

In [5]:
def term_by_document_matrix():
    files = os.listdir('sample_text/')
    t_b_d_m = []
    for i in range(1, 1101):
        file_name = f'sample_text/text_{i}.txt'
        bow_vector=bag_of_words_vector(file_name)
        t_b_d_m.append(bow_vector)
    return t_b_d_m

In [6]:
term_by_document_mtx = term_by_document_matrix()


In [7]:
def get_count_of_word(word):
    c = 0
    for i in term_by_document_mtx:
        if i.get(word)>0:
            c+=1
    return c

In [8]:
def multiply_by_IDF():
    for word in dictionary.keys():
        n_w = get_count_of_word(word)
        m = math.log(len(term_by_document_mtx)/n_w)
        for dic in term_by_document_mtx:
            dic[word] = dic.get(word)*m

In [9]:
multiply_by_IDF()


In [10]:
def create_vector(data):
    vector_dict={}
    for w in dictionary.keys():
        vector_dict[w]=0
    data = data.lower()
    data = re.sub(r'[^\w\s]','',data)
    data = re.sub('[0-9]','',data)
    words=nltk.word_tokenize(data)
    for w in words:
        if w in vector_dict:
            vector_dict[w]+=1
    return list(vector_dict.values())

def get_closest_documents(data,k):
    data_vect = create_vector(data)
    holder = 0
    index = None
    to_return = []
    for idx,i in enumerate(term_by_document_mtx):
        cos = np.matmul(np.array(data_vect).T,np.array(list(i.values())))/(np.linalg.norm(data_vect)*np.linalg.norm(list(i.values())))
        if len(to_return)<k:
            to_return.append((idx,cos))
            to_return = sorted(to_return, key=lambda x: x[1])
        else:
            if cos > to_return[0][1] or math.isnan(to_return[0][1]):
                to_return[0] = (idx,cos)
                to_return = sorted(to_return, key=lambda x: x[1])
    return to_return


        


In [11]:
t = get_closest_documents("And extends industry awards, the juno awards, which were as important as the quilombo of. Was down and hemp plantations. these men.. Spaces is terrain. unlike most mammals, when cats bring. History, 1815-1970. australia (6.4 percent), saudi. Verification of savannah from the swampland, were widespread during the nonbreeding. Monarch of classroom. in 2013, the beach handball world championships. News media regional airport, bert mooney airport and the plant and animal species. Kamerun. later, basin, red lodge, and whitefish mountain resort near libby whitefish. Fever were extreme emotional."
                          ,3)
def read_file(index):
    with io.open(f"sample_text/text_{index+1}.txt", "r") as f:
        print("______________________________________")
        print(f.read())
        print("______________________________________")

print(t)
for (idx,cos) in t:
    read_file(idx)

[(318, 0.08172827634973914), (702, 0.1519243455746266), (8, 0.6824081772328199)]
______________________________________
An old populism, that were. To language. golbery. with the implementation of classical music.. Portal climate oil deposits were formed in the. And are this definition. Or kingdoms, broadly to include certain types of. Are Ă©dith on seattle in 1941. this left an extensive. Kalahari desert river, then the world's. Though whether germans live abroad. jews are. Asia. colonialism plantations. these men, women and could not obtain identification and leave. Which winds morelos, who occupied. Most ancient including rainier beach, van asselt, rainier, and jefferson south of. Through e-mail. and 1940s..
______________________________________
______________________________________
Precipitating deck and whitefish mountain resort near red lodge. Festivals, colonial lakes account for news. By aristippus other types. for example, the. Hadron and 0.14%. 
 
 the world of coca-cola, f

In [12]:
from sklearn.preprocessing import normalize

In [13]:
def normalize_matrix():
    to_return = []
    for idx,i in enumerate(term_by_document_mtx):
        to_return.append(normalize([list(i.values())],norm="l1"))
    return to_return
    

In [14]:
normalized_A = normalize_matrix()

In [18]:
def get_cos_vector(data):
    data_vector_normalized = normalize([create_vector(data)],norm="l1")
    to_return = []
    for idx,i in enumerate(normalized_A):
        cos = np.matmul(np.array(data_vector_normalized[0]).T,np.array(i[0]))/(np.linalg.norm(data_vector_normalized)*np.linalg.norm(i))
        to_return.append((cos,idx))
    return to_return

    

In [35]:
data = "And extends industry awards, the juno awards, which were as important as the quilombo of. Was down and hemp plantations. these men.. Spaces is terrain. unlike most mammals, when cats bring. History, 1815-1970. australia (6.4 percent), saudi. Verification of savannah from the swampland, were widespread during the nonbreeding. Monarch of classroom. in 2013, the beach handball world championships. News media regional airport, bert mooney airport and the plant and animal species. Kamerun. later, basin, red lodge, and whitefish mountain resort near libby whitefish. Fever were extreme emotional. "
def get_closest_documents_normalized(data,k):
    cos_vector = get_cos_vector(data)
    cos_vector = sorted(cos_vector, reverse=True)
    return cos_vector[:k]

In [38]:
cld = get_closest_documents_normalized(data,3)
for doc,i in cld:
    read_file(i)

______________________________________
And extends industry awards, the juno awards, which were as important as the quilombo of. Was down and hemp plantations. these men.. Spaces is terrain. unlike most mammals, when cats bring. History, 1815-1970. australia (6.4 percent), saudi. Verification of savannah from the swampland, were widespread during the nonbreeding. Monarch of classroom. in 2013, the beach handball world championships. News media regional airport, bert mooney airport and the plant and animal species. Kamerun. later, basin, red lodge, and whitefish mountain resort near libby whitefish. Fever were extreme emotional.
______________________________________
______________________________________
Precipitating deck and whitefish mountain resort near red lodge. Festivals, colonial lakes account for news. By aristippus other types. for example, the. Hadron and 0.14%. 
 
 the world of coca-cola, featuring.
______________________________________
____________________________________

In [197]:
def svd_and_low_rank_approx(k):
    A = []
    for a in normalized_A:
        A.append(a[0])

    import scipy
    u,s,vt = scipy.sparse.linalg.svds(np.array(A),k=k)
    return u @ np.diag(s)@vt
    return result


In [198]:
# result = svd_and_low_rank_approx(100)
# print(result[0])

In [199]:
def get_after_approx(data,k):
    result = svd_and_low_rank_approx(k)
    data_vector_normalized = normalize([create_vector(data)],norm="l1")
    to_return = []
    for idx,i in enumerate(result):
        cos = np.matmul(np.array(data_vector_normalized[0]).T,np.array(i))/(np.linalg.norm(data_vector_normalized)*np.linalg.norm(i))
        to_return.append((cos,idx))
    return to_return

In [200]:
def get_closest_documents_after_approx(data,k,approx_k):
    cos_vector = get_after_approx(data,approx_k)
    cos_vector = sorted(cos_vector, reverse=True)
    return cos_vector[:k]


In [202]:
get_closest_documents_after_approx(data,3,200)

[(0.31925108385109424, 8),
 (0.22600177894809678, 508),
 (0.21714337518247243, 702)]